In [2]:
%pip install duckduckgo-search -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 59.4 MB/s eta 0:00:00


In [3]:
%pip install wikipedia -q

  Preparing metadata (setup.py) ... done


In [4]:
# Install required libraries
!pip install -q google-generativeai langchain-community langchain-openai langchain-anthropic pydantic gradio
!pip install langchain-google-genai -q
!pip install langchain-google-genai -q

# Import necessary libraries
from google.colab import userdata
import google.generativeai as genai
from langchain_community.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import Tool
from datetime import datetime
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_google_genai import ChatGoogleGenerativeAI
import gradio as gr

# Retrieve the API key from Colab Secrets
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

# Define tools
def save_to_txt(data: str, filename: str = "research_output.txt"):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    formatted_text = f"--- Research Output ---\nTimestamp: {timestamp}\n\n{data}\n\n"
    with open(filename, "a", encoding="utf-8") as f:
        f.write(formatted_text)
    return f"Data successfully saved to {filename}"

save_tool = Tool(
    name="save_text_to_file",
    func=save_to_txt,
    description="Saves structured research data to a text file.",
)

search = DuckDuckGoSearchRun()
search_tool = Tool(
    name="search",
    func=search.run,
    description="Search the web for information",
)

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

# Define Pydantic model
class ResearchResponse(BaseModel):
    topic: str
    summary: str
    sources: list[str]
    tools_used: list[str]

# Initialize the Google Generative AI model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=api_key)

# Set up parser and prompt
parser = PydanticOutputParser(pydantic_object=ResearchResponse)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a research assistant that will help generate a research paper.
            Answer the user query and use necessary tools.
            Wrap the output in this format and provide no other text\n{format_instructions}
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{query}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

# Create agent and executor
tools = [search_tool, wiki_tool, save_tool]
agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt,
    tools=tools
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Define Gradio interface function
def research_agent(query):
    raw_response = agent_executor.invoke({"query": query})
    try:
        structured_response = parser.parse(raw_response.get("output")[0]["text"])
        return f"Topic: {structured_response.topic}\nSummary: {structured_response.summary}\nSources: {', '.join(structured_response.sources)}\nTools Used: {', '.join(structured_response.tools_used)}"
    except Exception as e:
        return f"Error parsing response: {str(e)}\nRaw Response: {raw_response}"

# Launch Gradio interface
interface = gr.Interface(
    fn=research_agent,
    inputs="text",
    outputs="text",
    title="Research Assistant",
    description="Enter a research topic to get a summary and save it to a file.",
    theme="default"
)

interface.launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.9 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a1a5e61a14f4e71a54.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in t

In [ ]:
# Install required libraries
!pip install -q google-generativeai langchain-community langchain-openai langchain-anthropic pydantic gradio
!pip install langchain-google-genai -q
!pip install langchain-google-genai -q

# Import necessary libraries
from google.colab import userdata
import google.generativeai as genai
from langchain_community.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import Tool
from datetime import datetime
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_google_genai import ChatGoogleGenerativeAI
import gradio as gr

# Retrieve the API key from Colab Secrets
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

# Define tools
def save_to_txt(data: str, filename: str = "research_output.txt"):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    formatted_text = f"--- Research Output ---\nTimestamp: {timestamp}\n\n{data}\n\n"
    with open(filename, "a", encoding="utf-8") as f:
        f.write(formatted_text)
    return f"Data successfully saved to {filename}"

save_tool = Tool(
    name="save_text_to_file",
    func=save_to_txt,
    description="Saves structured research data to a text file.",
)

search = DuckDuckGoSearchRun()
search_tool = Tool(
    name="search",
    func=search.run,
    description="Search the web for information",
)

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

# Define Pydantic model
class ResearchResponse(BaseModel):
    topic: str
    summary: str
    sources: list[str]
    tools_used: list[str]

# Initialize the Google Generative AI model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=api_key)

# Set up parser and prompt
parser = PydanticOutputParser(pydantic_object=ResearchResponse)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a research assistant that will help generate a research paper.
            Answer the user query and use necessary tools.
            Wrap the output in this format and provide no other text\n{format_instructions}
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{query}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

# Create agent and executor
tools = [search_tool, wiki_tool, save_tool]
agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt,
    tools=tools
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Define Gradio interface function
def research_agent(query):
    raw_response = agent_executor.invoke({"query": query})
    try:
        structured_response = parser.parse(raw_response.get("output")[0]["text"])
        return f"Topic: {structured_response.topic}\nSummary: {structured_response.summary}\nSources: {', '.join(structured_response.sources)}\nTools Used: {', '.join(structured_response.tools_used)}"
    except Exception as e:
        return f"Error parsing response: {str(e)}\nRaw Response: {raw_response}"

# Custom CSS with Goku background
custom_css = """
body {
    background-image: url('https://example.com/goku_background.jpg'); /* Replace with actual URL or local path */
    background-size: cover;
    background-position: center;
    background-repeat: no-repeat;
    color: white; /* Ensure text is readable */
}
.gr-box { background-color: rgba(0, 0, 0, 0.7); } /* Semi-transparent background for input/output boxes */
.gr-button { color: white; } /* Ensure button text is readable */
"""

# Launch Gradio interface with custom CSS
interface = gr.Interface(
    fn=research_agent,
    inputs="text",
    outputs="text",
    title="Research Assistant",
    description="Enter a research topic to get a summary and save it to a file.",
    theme="default",
    css=custom_css
)

interface.launch(share=True)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.9 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5680be1303245ac6d2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co